In [1]:
import requests
import time
import numpy as np
import uuid
import shutil
import yaml

Define some useful functions and the addresses of the bServer and mServer globally

In [2]:
bServer = "http://127.0.0.1:18085/SIS/"
mServer = "http://127.0.0.1:18080"

In [3]:
def create_UUID():
    """UUIDs will be unique for a given run of the kernel"""
    unique_uuid = uuid.uuid4().hex[:8].upper()
    return unique_uuid


def create_SPECnewfile(new_name):
    t0=time.time()
    
    wait_until_SPECfinished()
    
    payload = {'spec_cmd': 'newfile {}'.format(new_name)}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent

def cam_refresh():
    t0=time.time()
    
    wait_until_SPECfinished()
    
    payload = {'spec_cmd': 'camrefresh'}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    time.sleep(45)
    wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent


def take_pilatus_image(exp_time, move_image_to=None):
    wait_until_SPECfinished()
    
    
    payload = {'spec_cmd': 'loopscan 1 {}'.format(exp_time)}
    r = requests.get(bServer + "execute_command", params=payload)
    
    wait_until_SPECfinished()

    counters = read_counters()
    
    image_savedpath = None
    if move_image_to is not None:
        pass
        
    
    return counters

def move_SPECmotor(motor_name, value, wait_until_finished = True):
    t0=time.time()
    payload = {'spec_cmd': 'mv {} {}'.format(motor_name, value)}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    if wait_until_finished is True:
        wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent

def set_SPECshutter(shutterOpen):
    t0=time.time()
    if shutterOpen is True:
        payload = {'spec_cmd': 'sopen'}
    else:
        payload = {'spec_cmd': 'sclose'}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    time.sleep(0.1)
    
    time_spent = time.time() - t0
    return time_spent

def wait_until_SPECfinished():
    t0 = time.time()
    while True:
        r = requests.get(bServer + "is_busy")
        is_busy = r.json()['data']
            
        if is_busy is False:
            break
    return time.time() - t0

def read_counters(save_to_file=None):
    t0 = time.time()
    
    #Get the mnemonics - a little wasteful but that is ok I think right now.
    r = requests.get(bServer + "get_all_counter_mnemonics", params={})
    counter_mnemonics = r.json()['data']

    wait_until_SPECfinished()
    
    
    r = requests.get(bServer + "get_all_counters", params={})
    counter_values = r.json()['data']
   
    time_spent = time.time() - t0
    
    return dict(zip(counter_mnemonics, counter_values))


def move_to_ROSSindex(ross_index):
    r = requests.post("http://127.0.0.1:18080/combined_move/move_to_index/{}".format(ross_index))
    #print(r)
    #print(r.json())

In [4]:
g = read_counters()
print(g)

{'sec': 0.2, 'mon': 0.0004, 'det': 0.0, 'dead': 0.0, 'i1': 8e-05, 'i2': 0.00189, 'i3': 0.0, 'i4': 0.0, 'normlz': 0.0, 'fstat': 0.0, 'TEMP': 0.0, 'pd1': -4.0, 'pd2': 0.0, 'pd3': 0.0, 'pd4': 0.0, 'pd5': 0.0, 'pd6': 0.0, 'pd7': 0.0, 'pd8': 0.0, 'pd9': 0.0, 'pd10': 0.0, 'pd11': 0.0, 'pd12': 0.0, 'pd13': 0.0, 'pd14': 0.0, 'pd15': 0.0, 'pd16': 0.0, 'ICR': 0.0, 'OCR': 0.0, 'ROI1': 0.0, 'ROI2': 0.0, 'ROI3': 0.0, 'ROI4': 0.0, 'keith_I': 0.0, 'keith_V': 0.0, 'CTEMP': 0.0, 'timesec': 0.0, 'tempsec': 0.0, 'PD_MIN': 0.0, 'PD_MAX': 0.0, 'A34410': 0.0, 'agdaq1': 0.0, 'agdaq2': 0.0, 'agdaq3': 0.0, 'agdaq4': 0.0, 'CTEMPB': 0.0}


In [16]:
cam_refresh()

45.07417416572571

In [5]:
move_SPECmotor('spr3', 51.1)

0.637932538986206

In [7]:
move_to_ROSSindex(2)

# Lets do some real work.... starting with the parameters defining the run.

Define the run name

In [17]:
scan_base_name = 'insitu_C2_thirdPressureSet'
path_to_detector = ""
path_to_saved_data = ""



Define the sample locations

In [20]:
sample_motors = {'x': 'spr2', 'y': 'spr3'}

sample_ROSSindex = {'QR1': "1", 'QR3': "0", 'air': "2"}
#Offset positions 
sample_global_offsets = {'QR1': {'x':47, 'y':50.50}, 'QR3': {'x':47, 'y':51.50}, 'air': {'x':49, 'y':52.50}} #Pressure set 3
#sample_global_offsets = {'QR1': {'x':47, 'y':50.25}, 'QR3': {'x':47, 'y':51.25}, 'air': {'x':49, 'y':52.25}} #Pressure set 2
#sample_global_offsets = {'QR1': {'x':47, 'y':50}, 'QR3': {'x':47, 'y':51}, 'air': {'x':49, 'y':52}}


exposure_time = 5
cam_refresh_freq = 10 #Number of scans before calling a cam refresh

Generate the positions to scan for each sample

In [21]:
sample_positions = {'QR1': [], 'QR3': [], 'air': []}


for rel_x_pos in np.linspace(0, 5, 5):
    for rel_y_pos in np.linspace(0, 5, 5):
        sample_positions['QR1'].append({'x': rel_x_pos + sample_global_offsets['QR1']['x'], 'y': rel_y_pos + sample_global_offsets['QR1']['y']})
        sample_positions['QR3'].append({'x': rel_x_pos + sample_global_offsets['QR3']['x'], 'y': rel_y_pos + + sample_global_offsets['QR3']['y']})
        
sample_positions['air'].append({'x': 0 + sample_global_offsets['air']['x'], 'y': 0 + sample_global_offsets['air']['y']})


#Rebuild the sample_scan_list. This is a bit more flexible way of constructing scans (e.g., you can interleave the samples or scan over positions in the sample... up to you)

sample_scan_list = []
sample_scan_list.append( {'QR1': sample_positions['QR1']})
sample_scan_list.append( {'QR3': sample_positions['QR3']})
sample_scan_list.append( {'air': sample_positions['air']})

print(sample_scan_list)

[{'QR1': [{'x': 47.0, 'y': 50.5}, {'x': 47.0, 'y': 51.75}, {'x': 47.0, 'y': 53.0}, {'x': 47.0, 'y': 54.25}, {'x': 47.0, 'y': 55.5}, {'x': 48.25, 'y': 50.5}, {'x': 48.25, 'y': 51.75}, {'x': 48.25, 'y': 53.0}, {'x': 48.25, 'y': 54.25}, {'x': 48.25, 'y': 55.5}, {'x': 49.5, 'y': 50.5}, {'x': 49.5, 'y': 51.75}, {'x': 49.5, 'y': 53.0}, {'x': 49.5, 'y': 54.25}, {'x': 49.5, 'y': 55.5}, {'x': 50.75, 'y': 50.5}, {'x': 50.75, 'y': 51.75}, {'x': 50.75, 'y': 53.0}, {'x': 50.75, 'y': 54.25}, {'x': 50.75, 'y': 55.5}, {'x': 52.0, 'y': 50.5}, {'x': 52.0, 'y': 51.75}, {'x': 52.0, 'y': 53.0}, {'x': 52.0, 'y': 54.25}, {'x': 52.0, 'y': 55.5}]}, {'QR3': [{'x': 47.0, 'y': 51.5}, {'x': 47.0, 'y': 52.75}, {'x': 47.0, 'y': 54.0}, {'x': 47.0, 'y': 55.25}, {'x': 47.0, 'y': 56.5}, {'x': 48.25, 'y': 51.5}, {'x': 48.25, 'y': 52.75}, {'x': 48.25, 'y': 54.0}, {'x': 48.25, 'y': 55.25}, {'x': 48.25, 'y': 56.5}, {'x': 49.5, 'y': 51.5}, {'x': 49.5, 'y': 52.75}, {'x': 49.5, 'y': 54.0}, {'x': 49.5, 'y': 55.25}, {'x': 49.5, 

Here we interleave some of the measurements between the two batteries to improve time resolution. (Not using this)

sample_positions = {'QR1': [], 'QR3': [], 'air': []}

sample_scan_list = []

for rel_x_pos in np.linspace(0, 5, 5):
    
    sample_positions['QR1'] = []
    sample_positions['QR3'] = []
    
    for rel_y_pos in np.linspace(0, 5, 10):
        sample_positions['QR1'].append({'x': rel_x_pos + sample_global_offsets['QR1']['x'], 'y': rel_y_pos + sample_global_offsets['QR1']['y']})
        sample_positions['QR3'].append({'x': rel_x_pos + sample_global_offsets['QR3']['x'], 'y': rel_y_pos + + sample_global_offsets['QR3']['y']})
        
    sample_scan_list.append( {'QR1': sample_positions['QR1']})
    sample_scan_list.append( {'QR3': sample_positions['QR3']})
    

sample_positions['air'].append({'x': 0 + sample_global_offsets['air']['x'], 'y': 0 + sample_global_offsets['air']['y']})

sample_scan_list.append( {'air': sample_positions['air']})

print(sample_scan_list)

Take control of SPEC

In [17]:
r = requests.get(bServer + "get_remote_control")
print(r.json())

{'help': 'Get the current plot. Not sure what this actually is', 'data': 'client in control.'}


## Ok. Lets do the actual scanning

In [22]:
#These previous values are used to prevent unnecessary motor movements in long scans. Set to None in the beginning to force the first move.
prev_ROSSindex = None
prev_x_pos = None
prev_y_pos = None

ordering_index = 0
do_cam_refresh = True
for scan_ctr in range(20000): #Set number scans to something large?
    tscan = time.time()
    
    #Check if we need to do a cam refresh
    if do_cam_refresh is True:
        if scan_ctr % cam_refresh_freq == 0: 
            print("Doing a cam refresh")
            cam_refresh()
    else:
        print("RE ENABLE CAM REFRESH checking!!!!!")    
    
    #Loop through each elemnt in the sample scan_list
    for single_scan_list in sample_scan_list:
        
        #Extract the sample name and the positions over which to scan
        for (sample_name, sample_pos_list) in single_scan_list.items():
            

            #Move to the right position on the ROSS switcher
            new_ROSSindex = sample_ROSSindex[sample_name]
            if prev_ROSSindex != new_ROSSindex:
                move_to_ROSSindex(new_ROSSindex)
                prev_ROSSindex = new_ROSSindex

            #Make a new SPEC file for each scan
            scan_SPECfilename = "{}_i{}_loop{}_{}".format(scan_base_name, ordering_index, scan_ctr, sample_name)
            ordering_index = ordering_index + 1
            print("Creating new SPEC file: {}".format(scan_SPECfilename))
            create_SPECnewfile(scan_SPECfilename)

            
            #Loop through the positions on the sample and take data
            pos_ctr = 0
            for xy_pos in sample_pos_list:

                #Compute the coordinates
                x_pos = xy_pos['x']
                y_pos = xy_pos['y']

                #Move the SPEC motors to the correct positions
                tSPECmove = time.time()
                
                #Check if we actually need to move x
                if prev_x_pos != x_pos:
                    move_SPECmotor(sample_motors['x'], x_pos)
                    prev_x_pos = x_pos

                #Check if we actually need to move y
                if prev_y_pos != y_pos:
                    move_SPECmotor(sample_motors['y'], y_pos)
                    prev_y_pos = y_pos

                print("{} pos {} ({},{}): move took {}s".format(sample_name, pos_ctr, x_pos, y_pos, time.time() - tSPECmove))



                #Take data
                f_name = "{}_{}_p{}_sc{}".format(scan_base_name, sample_name, pos_ctr, scan_ctr)
                take_pilatus_image(exposure_time)

                

                pos_ctr = pos_ctr + 1 #Increment the position counter

    print("Scan {} finished in {}s".format(scan_ctr, time.time() - tscan))
    #time.sleep(240.0) #wait?


            
        
        
        
        

Doing a cam refresh
Creating new SPEC file: insitu_C2_thirdPressureSet_i0_loop0_QR1
QR1 pos 0 (47.0,50.5): move took 0.7955875396728516s
QR1 pos 1 (47.0,51.75): move took 0.35997843742370605s
QR1 pos 2 (47.0,53.0): move took 0.3499794006347656s
QR1 pos 3 (47.0,54.25): move took 0.34999752044677734s
QR1 pos 4 (47.0,55.5): move took 0.3499767780303955s
QR1 pos 5 (48.25,50.5): move took 1.339919090270996s
QR1 pos 6 (48.25,51.75): move took 0.3499612808227539s
QR1 pos 7 (48.25,53.0): move took 0.35996294021606445s
QR1 pos 8 (48.25,54.25): move took 0.3499617576599121s
QR1 pos 9 (48.25,55.5): move took 0.3499789237976074s
QR1 pos 10 (49.5,50.5): move took 1.1481595039367676s
QR1 pos 11 (49.5,51.75): move took 0.3499784469604492s
QR1 pos 12 (49.5,53.0): move took 0.3499796390533447s
QR1 pos 13 (49.5,54.25): move took 0.3399960994720459s
QR1 pos 14 (49.5,55.5): move took 0.3499789237976074s
QR1 pos 15 (50.75,50.5): move took 1.1599478721618652s
QR1 pos 16 (50.75,51.75): move took 0.3499784469

QR3 pos 13 (49.5,55.25): move took 0.3499627113342285s
QR3 pos 14 (49.5,56.5): move took 0.3399794101715088s
QR3 pos 15 (50.75,51.5): move took 1.3499023914337158s
QR3 pos 16 (50.75,52.75): move took 0.35997700691223145s
QR3 pos 17 (50.75,54.0): move took 0.3399975299835205s
QR3 pos 18 (50.75,55.25): move took 0.339996337890625s
QR3 pos 19 (50.75,56.5): move took 0.34999680519104004s
QR3 pos 20 (52.0,51.5): move took 1.1599462032318115s
QR3 pos 21 (52.0,52.75): move took 0.3499622344970703s
QR3 pos 22 (52.0,54.0): move took 0.35999488830566406s
QR3 pos 23 (52.0,55.25): move took 0.3399624824523926s
QR3 pos 24 (52.0,56.5): move took 0.3499770164489746s
Creating new SPEC file: insitu_C2_thirdPressureSet_i8_loop2_air
air pos 0 (49,52.5): move took 1.199925184249878s
Scan 2 finished in 318.9861419200897s
Creating new SPEC file: insitu_C2_thirdPressureSet_i9_loop3_QR1
QR1 pos 0 (47.0,50.5): move took 0.7939419746398926s
QR1 pos 1 (47.0,51.75): move took 0.35997772216796875s
QR1 pos 2 (47.0,

QR1 pos 24 (52.0,55.5): move took 0.3499736785888672s
Creating new SPEC file: insitu_C2_thirdPressureSet_i16_loop5_QR3
QR3 pos 0 (47.0,51.5): move took 1.4077122211456299s
QR3 pos 1 (47.0,52.75): move took 0.3499619960784912s
QR3 pos 2 (47.0,54.0): move took 0.3499627113342285s
QR3 pos 3 (47.0,55.25): move took 0.35997915267944336s
QR3 pos 4 (47.0,56.5): move took 0.34999513626098633s
QR3 pos 5 (48.25,51.5): move took 1.1499314308166504s
QR3 pos 6 (48.25,52.75): move took 0.3499772548675537s
QR3 pos 7 (48.25,54.0): move took 0.35997819900512695s
QR3 pos 8 (48.25,55.25): move took 0.34998154640197754s
QR3 pos 9 (48.25,56.5): move took 0.3399960994720459s
QR3 pos 10 (49.5,51.5): move took 1.1499128341674805s
QR3 pos 11 (49.5,52.75): move took 0.3499612808227539s
QR3 pos 12 (49.5,54.0): move took 0.3418924808502197s
QR3 pos 13 (49.5,55.25): move took 0.3499624729156494s
QR3 pos 14 (49.5,56.5): move took 0.32999730110168457s
QR3 pos 15 (50.75,51.5): move took 1.159912109375s
QR3 pos 16 (50

QR1 pos 9 (48.25,55.5): move took 0.34999513626098633s
QR1 pos 10 (49.5,50.5): move took 1.1599302291870117s
QR1 pos 11 (49.5,51.75): move took 0.3499603271484375s
QR1 pos 12 (49.5,53.0): move took 0.34999656677246094s
QR1 pos 13 (49.5,54.25): move took 0.3499791622161865s
QR1 pos 14 (49.5,55.5): move took 0.3499796390533447s
QR1 pos 15 (50.75,50.5): move took 1.1499130725860596s
QR1 pos 16 (50.75,51.75): move took 0.34999608993530273s
QR1 pos 17 (50.75,53.0): move took 0.3499791622161865s
QR1 pos 18 (50.75,54.25): move took 0.35996103286743164s
QR1 pos 19 (50.75,55.5): move took 0.35996031761169434s
QR1 pos 20 (52.0,50.5): move took 1.3399369716644287s
QR1 pos 21 (52.0,51.75): move took 0.3399801254272461s
QR1 pos 22 (52.0,53.0): move took 0.34999680519104004s
QR1 pos 23 (52.0,54.25): move took 0.3399834632873535s
QR1 pos 24 (52.0,55.5): move took 0.35997724533081055s
Creating new SPEC file: insitu_C2_thirdPressureSet_i25_loop8_QR3
QR3 pos 0 (47.0,51.5): move took 1.4070076942443848s


QR3 pos 22 (52.0,54.0): move took 0.3499784469604492s
QR3 pos 23 (52.0,55.25): move took 0.3499627113342285s
QR3 pos 24 (52.0,56.5): move took 0.35999536514282227s
Creating new SPEC file: insitu_C2_thirdPressureSet_i32_loop10_air
air pos 0 (49,52.5): move took 1.1932201385498047s
Scan 10 finished in 363.99613213539124s
Creating new SPEC file: insitu_C2_thirdPressureSet_i33_loop11_QR1
QR1 pos 0 (47.0,50.5): move took 0.7969019412994385s
QR1 pos 1 (47.0,51.75): move took 0.34999656677246094s
QR1 pos 2 (47.0,53.0): move took 0.33732080459594727s
QR1 pos 3 (47.0,54.25): move took 0.35999441146850586s
QR1 pos 4 (47.0,55.5): move took 0.3499784469604492s
QR1 pos 5 (48.25,50.5): move took 1.3399195671081543s
QR1 pos 6 (48.25,51.75): move took 0.3499794006347656s
QR1 pos 7 (48.25,53.0): move took 0.3399803638458252s
QR1 pos 8 (48.25,54.25): move took 0.3499600887298584s
QR1 pos 9 (48.25,55.5): move took 0.3299977779388428s
QR1 pos 10 (49.5,50.5): move took 1.1411049365997314s
QR1 pos 11 (49.5,

...

QR3 pos 21 (52.0,52.75): move took 0.3499791622161865s
QR3 pos 22 (52.0,54.0): move took 0.35996198654174805s
QR3 pos 23 (52.0,55.25): move took 0.3499624729156494s
QR3 pos 24 (52.0,56.5): move took 0.35999441146850586s
Creating new SPEC file: insitu_C2_thirdPressureSet_i407_loop135_air
air pos 0 (49,52.5): move took 1.2086803913116455s
Scan 135 finished in 319.0061011314392s
Creating new SPEC file: insitu_C2_thirdPressureSet_i408_loop136_QR1
QR1 pos 0 (47.0,50.5): move took 0.7888522148132324s
QR1 pos 1 (47.0,51.75): move took 0.35999536514282227s
QR1 pos 2 (47.0,53.0): move took 0.3455491065979004s
QR1 pos 3 (47.0,54.25): move took 0.35997819900512695s
QR1 pos 4 (47.0,55.5): move took 0.35997653007507324s
QR1 pos 5 (48.25,50.5): move took 1.3399028778076172s
QR1 pos 6 (48.25,51.75): move took 0.34998035430908203s
QR1 pos 7 (48.25,53.0): move took 0.35999441146850586s
QR1 pos 8 (48.25,54.25): move took 0.3499796390533447s
QR1 pos 9 (48.25,55.5): move took 0.34999680519104004s
QR1 pos 

QR3 pos 5 (48.25,51.5): move took 1.3399019241333008s
QR3 pos 6 (48.25,52.75): move took 0.3499629497528076s
QR3 pos 7 (48.25,54.0): move took 0.349977970123291s
QR3 pos 8 (48.25,55.25): move took 0.3499901294708252s
QR3 pos 9 (48.25,56.5): move took 0.3499786853790283s
QR3 pos 10 (49.5,51.5): move took 1.3399176597595215s
QR3 pos 11 (49.5,52.75): move took 0.3499631881713867s
QR3 pos 12 (49.5,54.0): move took 0.34999561309814453s
QR3 pos 13 (49.5,55.25): move took 0.3499784469604492s
QR3 pos 14 (49.5,56.5): move took 0.34999561309814453s
QR3 pos 15 (50.75,51.5): move took 1.3599178791046143s
QR3 pos 16 (50.75,52.75): move took 0.35997676849365234s
QR3 pos 17 (50.75,54.0): move took 0.3499789237976074s
QR3 pos 18 (50.75,55.25): move took 0.3499789237976074s
QR3 pos 19 (50.75,56.5): move took 0.34999585151672363s
QR3 pos 20 (52.0,51.5): move took 1.1499302387237549s
QR3 pos 21 (52.0,52.75): move took 0.3699936866760254s
QR3 pos 22 (52.0,54.0): move took 0.3399965763092041s
QR3 pos 23 (5

QR1 pos 15 (50.75,50.5): move took 1.1499311923980713s
QR1 pos 16 (50.75,51.75): move took 0.35997796058654785s
QR1 pos 17 (50.75,53.0): move took 0.3499948978424072s
QR1 pos 18 (50.75,54.25): move took 0.3542037010192871s
QR1 pos 19 (50.75,55.5): move took 0.339996337890625s
QR1 pos 20 (52.0,50.5): move took 1.1499290466308594s
QR1 pos 21 (52.0,51.75): move took 0.35997962951660156s
QR1 pos 22 (52.0,53.0): move took 0.34997987747192383s
QR1 pos 23 (52.0,54.25): move took 0.3399786949157715s
QR1 pos 24 (52.0,55.5): move took 0.3499619960784912s
Creating new SPEC file: insitu_C2_thirdPressureSet_i424_loop141_QR3
QR3 pos 0 (47.0,51.5): move took 1.3819711208343506s
QR3 pos 1 (47.0,52.75): move took 0.3499789237976074s
QR3 pos 2 (47.0,54.0): move took 0.339979887008667s
QR3 pos 3 (47.0,55.25): move took 0.3499617576599121s
QR3 pos 4 (47.0,56.5): move took 0.34999585151672363s
QR3 pos 5 (48.25,51.5): move took 1.1499125957489014s
QR3 pos 6 (48.25,52.75): move took 0.35997819900512695s
QR3 

Creating new SPEC file: insitu_C2_thirdPressureSet_i432_loop144_QR1
QR1 pos 0 (47.0,50.5): move took 0.8010420799255371s
QR1 pos 1 (47.0,51.75): move took 0.34999513626098633s
QR1 pos 2 (47.0,53.0): move took 0.349977970123291s
QR1 pos 3 (47.0,54.25): move took 0.35997867584228516s
QR1 pos 4 (47.0,55.5): move took 0.331434965133667s
QR1 pos 5 (48.25,50.5): move took 1.3599340915679932s
QR1 pos 6 (48.25,51.75): move took 0.3499791622161865s
QR1 pos 7 (48.25,53.0): move took 0.35999584197998047s
QR1 pos 8 (48.25,54.25): move took 0.3499794006347656s
QR1 pos 9 (48.25,55.5): move took 0.3499784469604492s
QR1 pos 10 (49.5,50.5): move took 1.3471436500549316s
QR1 pos 11 (49.5,51.75): move took 0.34999632835388184s
QR1 pos 12 (49.5,53.0): move took 0.34999513626098633s
QR1 pos 13 (49.5,54.25): move took 0.3499627113342285s
QR1 pos 14 (49.5,55.5): move took 0.35997819900512695s
QR1 pos 15 (50.75,50.5): move took 1.3499186038970947s
QR1 pos 16 (50.75,51.75): move took 0.3499782085418701s
QR1 po

KeyboardInterrupt: 

Release control of SPEC

In [ ]:
r = requests.get(bServer + "release_remote_control")
print(r.json())